In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [ ]:
def plot_series(time, series, date_string=None, lab=None, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end],format,label=lab)
    plt.xlabel("Time in Days {}".format(date_string))
    plt.ylabel("Total Confirmed Cases")
    plt.legend()
    plt.title("Comparison of Actual and Predicted Total Confirmed cases")
    plt.grid(True)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<90000):
            print("\nCancelling training!")
            self.model.stop_training = True

In [ ]:
callbacks = myCallback()

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1) 
    ds = tf.data.Dataset.from_tensor_slices(series) 
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True) 
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:])) 
    return ds.batch(batch_size).prefetch(1)

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
tf.keras.backend.set_floatx('float64')

## 1. National Level Forecasting

In [ ]:
confirmed=[]

In [ ]:
with open('nation_level_daily_latest.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        confirmed.append(float(row[2]))

In [ ]:
time=np.arange(0,len(confirmed))

In [ ]:
series = np.array(confirmed)

In [ ]:
plot_series(time, series)

In [ ]:
split_time = 134
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

In [ ]:
window_size = 32
batch_size = 128 
shuffle_buffer_size = 100
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(50, activation="relu", return_sequences=True),
  tf.keras.layers.LSTM(50, activation="relu", return_sequences=True),
  tf.keras.layers.Dense(50,activation="relu"), 
  tf.keras.layers.Dense(50,activation="relu"),   
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100)
])

In [ ]:
model.compile(loss="mse",
              optimizer="adam")
history = model.fit(train_set, epochs=50000, callbacks=[callbacks])

In [ ]:
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid, "June 13 to June 19","Actual")
plot_series(time_valid, rnn_forecast,"June 13 to June 19","Predicted")
plt.savefig("National.png")

In [ ]:
tf.keras.metrics.mean_absolute_percentage_error(x_valid, rnn_forecast).numpy()

## 2. State Level Forecasting

In [ ]:
state="MH"

In [ ]:
df=pd.read_csv('state_wise_daily_latest.csv')

In [ ]:
df_=df.groupby("Status")
data=df_.get_group("Confirmed")
data=data[state]

In [ ]:
series=data.values[::]
series=series.astype("float64")
series=np.cumsum(series)

In [ ]:
time=np.arange(0,len(series))

In [ ]:
plot_series(time, series)

In [ ]:
split_time = 91
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

In [ ]:
window_size = 32
batch_size = 128 
shuffle_buffer_size = 100
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(50, activation="relu", return_sequences=True),
  tf.keras.layers.LSTM(50, activation="relu", return_sequences=True),  
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100)
])

In [ ]:
model.compile(loss="mse",
              optimizer="adam")
history = model.fit(train_set, epochs=50000,callbacks=[callbacks])

In [ ]:
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid,"June 13 to June 19","Actual")
plot_series(time_valid, rnn_forecast, "June 13 to June 19","Predicted")
plt.savefig("{}.png".format(state))

In [ ]:
tf.keras.metrics.mean_absolute_percentage_error(x_valid, rnn_forecast).numpy()